### **ChatMessageHistory** is the most fundamental memory class. It's a simple wrapper around a list of messages. It doesn't perform any summarization or windowing

In [24]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()  
google_api_key = os.getenv("GOOGLE_API_KEY")
llm= ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)

### ChatMessageHistory()

In [16]:
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
history=ChatMessageHistory()

history.add_user_message("What is the capital of France?")
history.add_ai_message("The capital of France is Paris.")
print(history.messages)

history.add_user_message("What is the capital of Germany?")
history.add_ai_message("The capital of Germany is Berlin.")
print(history.messages[2:])

[HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={})]
[HumanMessage(content='What is the capital of Germany?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of Germany is Berlin.', additional_kwargs={}, response_metadata={})]


### ConversationBufferMemory()

In [27]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
memory = ConversationBufferMemory(return_messages=True)

prompt= ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant who remembers conversations."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

chain=(
    RunnablePassthrough.assign(        
        history=lambda x: memory.load_memory_variables({})["history"]
    )
    | prompt
    | llm
)
response=chain.invoke({"input": "What is the capital of France?"})
print(response)
memory.save_context(
    {"input": "What is the capital of France?"},
    {"output": response.content}
)
print(chain.invoke({"input": "What is the previous question?"}))

content='The capital of France is Paris.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--a15590ad-933e-45b9-b9d5-9cf328160140-0' usage_metadata={'input_tokens': 16, 'output_tokens': 8, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}}
content='The previous question was: "What is the capital of France?"' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--2ddd84b4-5f27-4f77-9fca-a26d4820a117-0' usage_metadata={'input_tokens': 29, 'output_tokens': 14, 'total_tokens': 43, 'input_token_details': {'cache_read': 0}}


### ConversationBufferWindowMemory():-> window memory <br> 
memory_window = ConversationBufferWindowMemory(k=2, return_messages=True)


### ConversationalSummaryMemory() periodic summarization <br>
ConversationalSummaryMemory(llm=llm,return_messages=True)

### ConversationalKGMemeory() Knowledge Graph Memory
Benefit: Can handle very long conversations efficiently by only providing relevant facts, not the entire history.<br>
Complexity: Requires an LLM for graph extraction and potentially a graph database for persistence.

# Integration of memory in chains
**RunnableWithMessageHistory()** automates, Loading and Saving of context <br>

The get_session_history function is crucial for managing separate histories per user or session. In a real application, store would be replaced by a database (like Firestore, as mentioned in the initial instructions, or Redis, etc.).

The config={"configurable": {"session_id": session_id}} parameter in invoke() is how you tell RunnableWithMessageHistory which conversation history to use.

You no longer manually call memory.save_context() or memory.load_memory_variables(); RunnableWithMessageHistory handles it automatically.



In [ ]:
from langchain_core.runnables import RunnableWithMessageHistory
store={}

def get_session(session_id:str)->ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

core_chain=(
    ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant who remembers conversations."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ])
    | llm
)

chain_with_memory=RunnableWithMessageHistory(
    core_chain,
    get_session_history=get_session,
    input_messages_key="input",
    history_messages_key="history"
)

session_1="123"

response = chain_with_memory.invoke(
    {"input": "What is the capital of France?"},
    config={"configurable": {"session_id": session_1}}
)
print(response)

response_2= chain_with_memory.invoke(
    {"input": "What is the previous question"},
    config={"configurable": {"session_id": session_1}}
)
print("\n",response_2)
session_2="456"
response_3 = chain_with_memory.invoke(
    {"input": "What is the previous question"},
    config={"configurable": {"session_id": session_2}}
)
print("\n",response_3)

content='The capital of France is Paris.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--58f27467-873c-4346-84a2-55d060fa2318-0' usage_metadata={'input_tokens': 16, 'output_tokens': 8, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}}

 content='The previous question was "What is the capital of France?"' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--0b4f8cb3-ce84-4e8c-9494-0331c337f9d4-0' usage_metadata={'input_tokens': 28, 'output_tokens': 13, 'total_tokens': 41, 'input_token_details': {'cache_read': 0}}

 content="I don't have record of a previous question in this current conversation. This is our first interaction!" additional_kwargs={} response_metadata={'prompt_feedback': {'block_rea